In [1]:
import pandas as pd
import datetime as dt

In [2]:
jan = pd.read_parquet('./fhv_tripdata_2021-01.parquet')

In [3]:
# number of records in jan data
jan.shape

(1154112, 7)

In [4]:

jan.loc[:,'duration'] = jan.dropOff_datetime - jan.pickup_datetime
jan['duration']= jan.duration.dt.total_seconds()/60
# average duration
jan.duration.mean()

19.1672240937939

In [5]:
jan = jan[(jan.duration>=1) & (jan.duration<=60) ]
#Number of records after dropping
jan.shape

(1109826, 8)

In [6]:
# fraction of missing data
jan.PUlocationID.isna().sum()/jan.shape[0]

0.8352732770722617

In [7]:
categorical = ["PUlocationID","DOlocationID"]
train_df = jan[categorical + ["duration"]]
train_df.fillna(-1, inplace = True)
train_df[categorical]= train_df[categorical].astype("str")
# X_train = pd.concat([pd.get_dummies(train_df['PUlocationID'], prefix='P'),  pd.get_dummies(train_df['DOlocationID'], prefix='D')],axis=1)  
# X_train.shape
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

dv = DictVectorizer()
X_train = train_df.iloc[:,:2]
y_train = train_df.iloc[:,2].values
X_train_dict = X_train.to_dict(orient = "records")
X_train = dv.fit_transform(X_train_dict)


/Users/chandra/miniconda3/envs/DL/lib/python3.7/site-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/chandra/miniconda3/envs/DL/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
y_train = train_df.iloc[:,2].values
lr = LinearRegression()
lr.fit(X_train,y_train)
#mse
y_pred = lr.predict(X_train)
mean_squared_error(y_train,y_pred, squared = False)

10.528519107205451

In [9]:
feb = pd.read_parquet('./fhv_tripdata_2021-02.parquet')
feb.loc[:,'duration'] = feb.dropOff_datetime - feb.pickup_datetime
feb['duration']= feb.duration.dt.total_seconds()/60
feb = feb[(feb.duration>=1) & (feb.duration<=60) ]
categorical = ["PUlocationID","DOlocationID"]
valid_df = feb[categorical + ["duration"]]
valid_df.fillna(-1, inplace = True)
valid_df[categorical]= valid_df[categorical].astype("str")
# X_valid = pd.concat([pd.get_dummies(train_df['PUlocationID'], prefix='P'),  pd.get_dummies(train_df['DOlocationID'], prefix='D')],axis=1)  
# y_valid = train_df.iloc[:,2].values




# dv = DictVectorizer()
X_valid = valid_df.iloc[:,:2]
y_valid = valid_df.iloc[:,2].values
X_valid_dict = X_valid.to_dict(orient = "records")
X_valid = dv.transform(X_valid_dict)

/Users/chandra/miniconda3/envs/DL/lib/python3.7/site-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/chandra/miniconda3/envs/DL/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [10]:
X_train.shape, X_valid.shape

((1109826, 525), (990113, 525))

In [11]:
# X_valid = X_valid[X_train.columns]

In [12]:
#mse
valid_pred = lr.predict(X_valid)

In [13]:
mean_squared_error(y_valid,valid_pred, squared = False)

11.014283139629091